# Part 1: Dataframe creation

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

In [3]:
BSoup = BeautifulSoup(source, 'xml')

In [4]:
BSoup

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ea534255-3e5f-4b70-b686-3ff030a74256","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":979555370,"wgRevisionId":979555370,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from W

In [5]:
table=BSoup.find('table')

## Creating dataframe with columns PostalCode, Borough, and Neighborhood

In [6]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
df

,Postalcode,Borough,Neighborhood


# Data Scraping

### Search all the postcode, borough, neighborhood 

In [7]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [8]:
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


# Data Cleaning 

In [9]:
df=df[df['Borough']!='Not assigned']


In [10]:
df.shape

(103, 3)

In [11]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [12]:
DFM = pd.merge(df, temp_df, on='Postalcode')

In [13]:
DFM.drop(['Neighborhood'],axis=1,inplace=True)

In [14]:
DFM.drop_duplicates(inplace=True)

In [15]:
DFM.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [16]:
DFM.head(20)


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [17]:
DFM.shape

(103, 3)

# Part 2 Getting geospatial data of neighborhoods

In [18]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')

In [19]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df_geo.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geomerge = pd.merge(df_geo, DFM, on='Postalcode')


In [21]:
geomerge

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [22]:
geo_data=geomerge[['Postalcode','Latitude','Longitude','Borough','Neighborhood']]

In [23]:
geo_data.head(20)

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,43.711112,-79.284577,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,43.716316,-79.239476,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West"


# Part 3 Exploring and clustering the neighborhoods in Toronto

In [24]:
Boroughs=geo_data['Borough'].unique()
Boroughs

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [25]:
Toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
Toronto_data.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
37,M4E,43.676357,-79.293031,East Toronto,The Beaches
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
42,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
43,M4M,43.659526,-79.340923,East Toronto,Studio District
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park


In [26]:
# @hidden_cell
CLIENT_ID = '4H5MRCQNNGOZTEQAW4GWJAJVBRWMNQV3BEZWIGUTSIUDPYLG' 
CLIENT_SECRET = 'KDKGNC0QDPDRTX2UH4MFR2QQ5ZX4CLP1V2NXPI15U02XQH3Y' 
VERSION = '20201219'

In [27]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        #1   
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # 2
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # 3
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [29]:
Toronto_venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
6,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
7,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677702,-79.352265,Ice Cream Shop
8,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
9,"The Danforth West, Riverdale",43.679557,-79.352188,The Big Carrot Organic Juice Bar,43.677438,-79.352683,Juice Bar


In [30]:
# one hot encoding
Toronto_ohc = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_ohc['Neighborhood'] = Toronto_venues['Neighborhood']
Toronto_ohc.shape

(1621, 232)

In [31]:
Toronto_group = Toronto_ohc.groupby('Neighborhood').mean().reset_index()
Toronto_group.head(10)

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,...,0.0000,0.0,0.000000,0.0,0.0,0.016393,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,...,0.0000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,...,0.0000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0000,0.0,...,0.0000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Central Bay Street,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,...,0.0000,0.0,0.000000,0.0,0.0,0.015385,0.0,0.0,0.015385,0.015385
5,Christie,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,...,0.0000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
6,Church and Wellesley,0.0125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0125,0.0,...,0.0125,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.025000
7,"Commerce Court, Victoria Hotel",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0300,0.0,...,0.0000,0.0,0.000000,0.0,0.0,0.020000,0.0,0.0,0.010000,0.000000
8,Davisville,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,...,0.0000,0.0,0.030303,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
9,Davisville North,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,...,0.0000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [32]:
def get_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# creating columns to the number of top venues
columns = ['Neighborhood']
for ven in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ven+1, indicators[ven]))
    except:
        columns.append('{}th Most Common Venue'.format(ven+1))

# create a new dataframe
venues_neighborhoods_sorted = pd.DataFrame(columns=columns)
venues_neighborhoods_sorted['Neighborhood'] = Toronto_group['Neighborhood']

for ven in np.arange(Toronto_group.shape[0]):
    venues_neighborhoods_sorted.iloc[ven, 1:] = get_most_common_venues(Toronto_group.iloc[ven, :], num_top_venues)

venues_neighborhoods_sorted.head(5)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Farmers Market,Seafood Restaurant,Cheese Shop,Beer Bar,Restaurant,Café
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Pizza Place,Light Rail Station,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Butcher,Restaurant,Auto Workshop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Bar
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bank,Salad Place,Thai Restaurant,Bubble Tea Shop,Burger Joint,Yoga Studio


### Clustering Neighborhoods

In [34]:
kclusters = 5

Toronto_group_cluster = Toronto_group.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_group_cluster)

kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 4], dtype=int32)

### Adding labels and coordinates

In [35]:
venues_neighborhoods_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

Toronto_merged = Toronto_merged.join(venues_neighborhoods_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged

,Postalcode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,2,Trail,Health Food Store,Pub,Yoga Studio,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",2,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Pub,Pizza Place,Lounge
42,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West",2,Fast Food Restaurant,Board Shop,Sushi Restaurant,Brewery,Sandwich Place,Liquor Store,Italian Restaurant,Burrito Place,Restaurant,Ice Cream Shop
43,M4M,43.659526,-79.340923,East Toronto,Studio District,2,Coffee Shop,Brewery,Gastropub,Bakery,Café,American Restaurant,Yoga Studio,Convenience Store,Cheese Shop,Pizza Place
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,0,Park,Swim School,Bus Line,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
45,M4P,43.712751,-79.390197,Central Toronto,Davisville North,4,Dance Studio,Department Store,Park,Gym / Fitness Center,Breakfast Spot,Sandwich Place,Food & Drink Shop,Hotel,Eastern European Restaurant,Donut Shop
46,M4R,43.715383,-79.405678,Central Toronto,"North Toronto West, Lawrence Park",2,Coffee Shop,Clothing Store,Yoga Studio,Sporting Goods Shop,Gym / Fitness Center,Fast Food Restaurant,Italian Restaurant,Diner,Mexican Restaurant,Cosmetics Shop
47,M4S,43.704324,-79.388790,Central Toronto,Davisville,2,Dessert Shop,Sandwich Place,Pizza Place,Sushi Restaurant,Gym,Italian Restaurant,Café,Coffee Shop,Toy / Game Store,Greek Restaurant
48,M4T,43.689574,-79.383160,Central Toronto,"Moore Park, Summerhill East",2,Tennis Court,Trail,Playground,Restaurant,Yoga Studio,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner
49,M4V,43.686412,-79.400049,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",2,Coffee Shop,Pizza Place,American Restaurant,Supermarket,Sushi Restaurant,Bank,Fried Chicken Joint,Restaurant,Bagel Shop,Pub


In [36]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [37]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters